In [1]:
# Smarking Data Transformation Task, presented by Suvro Banerjee

In [1]:
import pandas as pd
import datetime
import math

In [2]:
# Read the pickeled file (each row is a transaction record)
trans = pd.read_pickle('transactions.pkl')

In [3]:
trans.shape

(236855, 39)

In [4]:
trans[0:1]

,entry_time,entry_year,entry_month,entry_day,entry_hour,entry_minute,entry_dayofweek,entry_date,entry_date_hour,entry_year_week,...,exit_device,ticket_number,ticket_type,trans_number,trans_type,rate,card_number,first_name,last_name,access_group
0,2012-07-06 17:13:00,2012,7,6,17,13,Friday,2012-07-06,2012-07-06 17:00:00,201226,...,ACG Red T/C Exit 6800//L,25681,Entry,251,Normal,1,-1,NaN,NaN,Transient


In [5]:
trans.dtypes

entry_time          datetime64[ns]
entry_year                   int32
entry_month                  int32
entry_day                    int32
entry_hour                   int32
entry_minute                 int32
entry_dayofweek             object
entry_date                  object
entry_date_hour     datetime64[ns]
entry_year_week              int32
entry_year_month             int32
exit_time           datetime64[ns]
exit_year                    int32
exit_month                   int32
exit_day                     int32
exit_hour                    int32
exit_minute                  int32
exit_dayofweek              object
exit_date                   object
exit_date_hour      datetime64[ns]
exit_year_week               int32
exit_year_month              int32
duration_min                 int64
duration_hour              float64
duration_hour_up             int64
revenue                    float64
gross_revenue              float64
lot                          int64
entry_device        

In [5]:
# Let's take the relevant colum for this task
trans_project = trans.ix[:, ['entry_time', 'exit_time', 'ticket_number']]

In [6]:
trans_project[0:1]

,entry_time,exit_time,ticket_number
0,2012-07-06 17:13:00,2013-01-24 22:49:00,25681


In [7]:
# Now the logic begins, the intention is given below-
# Create a dictionary, say (dict-1) with key as the 'start_date'
# the value would be another dictionary, say (dict-2).
# The second dictionary dict-2 will have key as "duration" i.e. exit_time - entry_time
# and the value would be a counter that will have the number of cars.
# Let's do it now.

In [8]:
# This function computes the number of cars in specific interval of duration.
# Input : User input of "Start Time", "End Time", "Day of the Week" and "Hour of the Day"
# The parsed search string is the Input.
# Output: Returns a dictionary which contains the search result.

def computeNumCars(searchParking):
    
    entry_key = {}  # outer dict, holds key as "start_date" and value as another dict.
    
    for index, row in trans_project.iterrows():
        ent_time = row['entry_time'] # entry_time
        ext_time = row['exit_time'] # exit_time
        
        duration = ext_time - ent_time # duration of stay (timedelta64 type)
        duration_hr = math.ceil((duration.total_seconds()) / 3600) # duration in hour
        
        ent_str = ent_time.strftime('%Y/%m/%d %H') # converting entry_time to string
        
        # if the entry_time is a new time
        if ent_str not in entry_key:
            dur_key = {} # this dict is the value of the outer dict (entry_key)
            
            # duration_hr is the key and value is a counter set to 1 for the 1st time
            dur_key[duration_hr] = 1 
            
            entry_key[ent_str] = dur_key  # setting the value of the outer dict.
        
        # if the start-date already exists in the outer dict
        else:
            duration_map = entry_key.get(ent_str) # get the start date and store it in a map
            
            if duration_hr not in duration_map:
                duration_map[duration_hr] = 1
            else:
                num_of_cars = duration_map.get(duration_hr) + 1
                duration_map.update({duration_hr : num_of_cars})
                
    searchParking = str(searchParking)
    try:
        print('The Parking Place information on ', searchParking, 'is as below')
        return entry_key[searchParking]
    except KeyError:
        pass

In [9]:
number_to_day_of_week = {0: 'Monday',
                         1: 'Tuesday',
                         2: 'Wednesday',
                         3: 'Thursday',
                         4: 'Friday',
                         5: 'Saturday',
                         6: 'Sunday'}

day_of_week_to_number = {'Monday': 0,
                         'Tuesday': 1,
                         'Wednesday': 2,
                         'Thursday': 3,
                         'Friday': 4,
                         'Saturday': 5,
                         'Sunday': 6}

In [57]:
# Take the raw input
input_StrtDt = input("Enter the Start Date (YYYY/MM/DD HH:MM)    ")
input_EndDt = input("Enter the End Date (YYYY/MM/DD HH:MM)       ")
input_DayOfWk = input("Enter the Entry Day of Week            ")
input_HourOfDay = input("Enter the Entry Hour                      ")

# Setting Flags for scanarios with "All" 

if (input_DayOfWk.title() == 'All'):
    is_Day_All = True
else:
    is_Day_All = False
    
if (input_HourOfDay.title() == 'All'):
    isHourAll = True
else:
    isHourAll = False

# Converting the input to Timmestamp from string and doing transformations.

input_StrtDt_Date = datetime.datetime.strptime(input_StrtDt, "%Y/%m/%d %H:%M")  # convert start date into date
input_EndDt_Date = datetime.datetime.strptime(input_EndDt, "%Y/%m/%d %H:%M")    # convert the end date into date

input_StrtDt_Date_Weekday = datetime.datetime.weekday(input_StrtDt_Date) # Weekday (returns a number) from start date


# The following code is to set up the INITIAL date increments based on different scenarios.

if is_Day_All:
    pass # input_StrtDt_Date will remain the same when the start date and the day of the week is same.    

else:
    input_DayOfWk_Weekday = input_DayOfWk.title()  # proper case
    input_DayOfWk_Weekday_Map_Num = day_of_week_to_number[input_DayOfWk_Weekday] # returns a number corres to the Input Weekday
    
    # if input Day of Week is ahead of the input Start Date
    if (input_DayOfWk_Weekday_Map_Num > input_StrtDt_Date_Weekday):  # to set the increment of days
        days_diff = input_DayOfWk_Weekday_Map_Num - input_StrtDt_Date_Weekday
        input_StrtDt_Date += datetime.timedelta(days = days_diff)
    
    # if input Start Date is ahead of the input Day of the Week, in this case startdate to be incremented to the next week.
    elif (input_StrtDt_Date_Weekday > input_DayOfWk_Weekday_Map_Num):  # to set the increment of days
        days_diff_1 = 6 - input_StrtDt_Date_Weekday
        days_diff_2 = input_DayOfWk_Weekday_Map_Num - 0
        days_diff = days_diff_1 + days_diff_2 + 1
        input_StrtDt_Date += datetime.timedelta(days = days_diff)
        
    else:
        pass  # inputStrtDt_DT will remain the same when the start date and the day of the week is same.


while (input_StrtDt_Date <= input_EndDt_Date):  # to generate a sequence of dates 
    
    input_StrtDt_Date_Weekday_New = datetime.datetime.weekday(input_StrtDt_Date)   # Weekday (returns a number) from start date
    input_StrtDt_Date_Weekday_New_Map = number_to_day_of_week[input_StrtDt_Date_Weekday_New] # convert the number to weekday (using a dict)
    
    # only if the Day of the Week entered is equal to the day of the New start date (computed), here both are like "Wednesday" etc.
    if (input_DayOfWk_Weekday == input_StrtDt_Date_Weekday_New_Map and not is_Day_All):
        
        if isHourAll:
            parsed_SearchDate = datetime.datetime.strftime(input_StrtDt_Date, '%Y/%m/%d')
            hour = input_StrtDt_Date.hour
            
            if len(str(hour)) > 1:  # comes from input start date
                parsed_SearchDate += ' ' + str(hour)  # parsing the Hour of the day
            else:
                parsed_SearchDate += ' 0'+ str(hour)  # parsing the Hour of the day
            
            print(computeNumCars(parsed_SearchDate))
            
            input_StrtDt_Date += datetime.timedelta(hours = 1)  # increment by 1 hr as All hour and after 24 goes to the next day(which we'll handle below)
            if hour < 23:
                hour += 1
            else:
                hour = 0
                input_StrtDt_Date += datetime.timedelta(days = 6) # scenario when Day of week has a number and Hour of Day is All.
      
        else:
            # start creating the search string for the computeNumCars()
            parsed_SearchDate = datetime.datetime.strftime(input_StrtDt_Date, '%Y/%m/%d')
        
            if len(input_HourOfDay) > 1:
                parsed_SearchDate += ' ' + input_HourOfDay  # parsing the Hour of the day
            else:
                parsed_SearchDate += ' 0'+ input_HourOfDay  # parsing the Hour of the day with a zero, due to our dict index.
    
            print(computeNumCars(parsed_SearchDate))  # Printing the dictionary. (Key: Duration in Hour) & (Value: Number of Cars)
        
            input_StrtDt_Date += datetime.timedelta(days = 7) # increment by 7 days
            
        
        
        
    elif is_Day_All:  # All days
        
        if isHourAll:
            parsed_SearchDate = datetime.datetime.strftime(input_StrtDt_Date, '%Y/%m/%d')
            hour = input_StrtDt_Date.hour
            
            if len(str(hour)) > 1:  # comes from input start date
                parsed_SearchDate += ' ' + str(hour)  # parsing the Hour of the day
            else:
                parsed_SearchDate += ' 0'+ str(hour)  # parsing the Hour of the day
            
            print(computeNumCars(parsed_SearchDate))
            
            input_StrtDt_Date += datetime.timedelta(hours = 1)  # increment by 1 hr as All hour and after 24 goes to the next day(which is desirable.)
            
        
        else:
            # start creating the search string for the computeNumCars()
            parsed_SearchDate = datetime.datetime.strftime(input_StrtDt_Date, '%Y/%m/%d')
            
            if len(input_HourOfDay) > 1:
                parsed_SearchDate += ' ' + input_HourOfDay  # parsing the Hour of the day
            else:
                parsed_SearchDate += ' 0'+ input_HourOfDay  # parsing the Hour of the day with a zero, due to our dict index.
    
            print(computeNumCars(parsed_SearchDate))  # Printing the dictionary. (Key: Duration in Hour) & (Value: Number of Cars)
        
            input_StrtDt_Date += datetime.timedelta(days = 1) # increment by 1 day

Enter the Start Date (YYYY/MM/DD HH:MM)    2013/06/07 06:00
Enter the End Date (YYYY/MM/DD HH:MM)       2013/06/27 10:00
Enter the Entry Day of Week            Friday
Enter the Entry Hour                      10
The Parking Place information on  2013/06/07 10 is as below
{1: 7, 2: 11, 3: 3, 4: 2, 5: 2, 6: 2, 7: 3}
The Parking Place information on  2013/06/14 10 is as below
{1: 6, 2: 6, 3: 5, 4: 2, 5: 2, 6: 2, 7: 2}
The Parking Place information on  2013/06/21 10 is as below
{1: 5, 2: 3, 3: 4, 4: 2, 5: 1, 6: 2, 8: 1}


In [ ]:
# The above is a result of one of the scenarios.